In [99]:
"""This script estimates TRFs for several models and saves them"""
from pathlib import Path
import re

In [100]:
import eelbrain
import mne

In [101]:

DATA_ROOT = Path("~").expanduser() / 'Data' / 'cocoha'
PREDICTOR_DIR = DATA_ROOT / 'predictors'
EEG_DIR = DATA_ROOT / 'eeg'
SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d+_data_preproc', path.name)]

# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs'
TRF_DIR.mkdir(exist_ok=True)

STIMULUS_DIR = DATA_ROOT / 'stimuli'
STIMULI_PATHS = [stimulus.stem for stimulus in STIMULUS_DIR.glob("*.wav")]

In [102]:
"""

# Load stimuli
# ------------
# Make sure to name the stimuli so that the TRFs can later be distinguished
# Load the gammatone-spectrograms; use the time axis of these as reference
gammatone = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-8.pickle') for path in STIMULI_PATHS]
# Resample the spectrograms to 100 Hz (time-step = 0.01 s), which we will use for TRFs
gammatone = [x.bin(0.01, dim='time', label='start') for x in gammatone]
# Pad onset with 100 ms and offset with 1 second; make sure to give the predictor a unique name as that will make it easier to identify the TRF later
gammatone = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='gammatone') for x in gammatone]
# Filter the predictor with the same parameters as we will filter the EEG data
gammatone = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone]

"""

"\n\n# Load stimuli\n# ------------\n# Make sure to name the stimuli so that the TRFs can later be distinguished\n# Load the gammatone-spectrograms; use the time axis of these as reference\ngammatone = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-8.pickle') for path in STIMULI_PATHS]\n# Resample the spectrograms to 100 Hz (time-step = 0.01 s), which we will use for TRFs\ngammatone = [x.bin(0.01, dim='time', label='start') for x in gammatone]\n# Pad onset with 100 ms and offset with 1 second; make sure to give the predictor a unique name as that will make it easier to identify the TRF later\ngammatone = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='gammatone') for x in gammatone]\n# Filter the predictor with the same parameters as we will filter the EEG data\ngammatone = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone]\n\n"

In [103]:
gammatone = {}
for path in STIMULI_PATHS:
    x = eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-8.pickle')
    x = x.bin(1/64, dim='time', label='start')
    x = eelbrain.pad(x, tstart=0, name='gammatone')
    x = eelbrain.filter_data(x, 0.5, 20)
    gammatone[path] = x
    

In [104]:
envelope = {}
onset_envelope = {}
gammatone_onsets = {}
gammatone_lin = {}
gammatone_pow = {}
word_onsets = {}
word_lexical = {}
word_nlexical = {}

for path in STIMULI_PATHS:
    
    x = eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-1.pickle')
    x = x.bin(1/64, dim='time', label='start')
    x = eelbrain.pad(x, tstart=0, name='envelope')
    x = eelbrain.filter_data(x, 0.5, 20)
    envelope[path] = x

    x_on = eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-on-1.pickle')
    x_on = x_on.bin(1/64, dim='time', label='start')
    x_on = eelbrain.pad(x_on, tstart=0, name='onset')
    x_on = eelbrain.filter_data(x_on, 0.5, 20)
    onset_envelope[path] = x_on

## Load the broad-band envelope and process it in the same way
#envelope = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-1.pickle') for path in STIMULI_PATHS]
#envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
#envelope = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
#envelope = [eelbrain.filter_data(x, 0.5, 20) for x in envelope]
#onset_envelope = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{path}~gammatone-on-1.pickle') for path in STIMULI_PATHS]
#onset_envelope = [x.bin(0.01, dim='time', label='start') for x in onset_envelope]
#onset_envelope = [eelbrain.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='onset') for x in onset_envelope]
#onset_envelope = [eelbrain.filter_data(x, 0.5, 20) for x in onset_envelope]

# WAIT FOR NOW
"""

# Load onset spectrograms and make sure the time dimension is equal to the gammatone spectrograms
gammatone_onsets = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-on-8.pickle') for stimulus in STIMULI]
gammatone_onsets = [x.bin(0.01, dim='time', label='start') for x in gammatone_onsets]
gammatone_onsets = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_onsets, gammatone)]
gammatone_onsets = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone_onsets]



# Load linear and powerlaw scaled spectrograms
gammatone_lin = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-lin-8.pickle') for stimulus in STIMULI]
gammatone_lin = [x.bin(0.01, dim='time', label='start') for x in gammatone_lin]
gammatone_lin = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_lin, gammatone)]
gammatone_lin = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone_lin]
gammatone_pow = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-pow-8.pickle') for stimulus in STIMULI]
gammatone_pow = [x.bin(0.01, dim='time', label='start') for x in gammatone_pow]
gammatone_pow = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_pow, gammatone)]
gammatone_pow = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone_pow]
# Load word tables and convert tables into continuous time-series with matching time dimension
word_tables = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~word.pickle') for stimulus in STIMULI]
word_onsets = [eelbrain.event_impulse_predictor(gt.time, data=data, name='word') for gt, data in zip(gammatone, word_tables)]
# Function and content word impulses based on the boolean variables in the word-tables
word_lexical = [eelbrain.event_impulse_predictor(gt.time, value='lexical', data=data, name='lexical') for gt, data in zip(gammatone, word_tables)]
word_nlexical = [eelbrain.event_impulse_predictor(gt.time, value='nlexical', data=data, name='non_lexical') for gt, data in zip(gammatone, word_tables)]

"""

"\n\n# Load onset spectrograms and make sure the time dimension is equal to the gammatone spectrograms\ngammatone_onsets = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-on-8.pickle') for stimulus in STIMULI]\ngammatone_onsets = [x.bin(0.01, dim='time', label='start') for x in gammatone_onsets]\ngammatone_onsets = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_onsets, gammatone)]\ngammatone_onsets = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone_onsets]\n\n\n\n# Load linear and powerlaw scaled spectrograms\ngammatone_lin = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-lin-8.pickle') for stimulus in STIMULI]\ngammatone_lin = [x.bin(0.01, dim='time', label='start') for x in gammatone_lin]\ngammatone_lin = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_lin, gammatone)]\ngammatone_lin = [eelbrain.filter_data(x, 0.5, 20) for x in gammatone_lin]\ngammatone_pow = [eelbrain.load.unpickle(PRED

In [105]:
# Models
# ------
# Pre-define models here to have easier access during estimation. In the future, additional models could be added here and the script re-run to generate additional TRFs.
models = {
    'envelope': envelope,
    # Compare different scales for the acoustic response
    'gammatone': gammatone,
    #'gammatone-lin': gammatone_lin,
    #'gammatone-pow': gammatone_pow,
    #'gammatone-lin+log': {
    #    'gammatone_lin': gammatone_lin, 
    #    'gammatone': gammatone
    #},
    # The acoustic edge detection model
    'envelope+onset': {
        'envelope': envelope,
        'onset': onset_envelope
    },
    #'acoustic': [gammatone, gammatone_onsets],
    # Models with word-onsets and word-class
    #'words': word_onsets,
    #'words+lexical': [word_onsets, word_lexical, word_nlexical],
    #'acoustic+words': [gammatone, gammatone_onsets, word_onsets],
    #'acoustic+words+lexical': [gammatone, gammatone_onsets, word_onsets, word_lexical, word_nlexical],
}

In [106]:
models

{'envelope': {'marianne_story5_trial_5': <NDVar 'marianne_story5_trial_5.wav': 3200 time>,
  'aske_story2_trial_3': <NDVar 'aske_story2_trial_3.wav': 3200 time>,
  'marianne_story4_trial_22': <NDVar 'marianne_story4_trial_22.wav': 3200 time>,
  'marianne_story4_trial_23': <NDVar 'marianne_story4_trial_23.wav': 3200 time>,
  'aske_story2_trial_2': <NDVar 'aske_story2_trial_2.wav': 3200 time>,
  'marianne_story5_trial_4': <NDVar 'marianne_story5_trial_4.wav': 3200 time>,
  'marianne_story5_trial_6': <NDVar 'marianne_story5_trial_6.wav': 3200 time>,
  'marianne_story4_trial_21': <NDVar 'marianne_story4_trial_21.wav': 3200 time>,
  'aske_story2_trial_11': <NDVar 'aske_story2_trial_11.wav': 3200 time>,
  'aske_story2_trial_10': <NDVar 'aske_story2_trial_10.wav': 3200 time>,
  'marianne_story4_trial_20': <NDVar 'marianne_story4_trial_20.wav': 3200 time>,
  'aske_story2_trial_1': <NDVar 'aske_story2_trial_1.wav': 3200 time>,
  'marianne_story5_trial_7': <NDVar 'marianne_story5_trial_7.wav': 3

In [107]:
expinfo_table = eelbrain.load.tsv("../expinfo.csv", encoding='utf-8-sig').as_dataframe()
expinfo_table.head()

,attend_mf,attend_lr,acoustic_condition,n_speakers,wavfile_male,wavfile_female,trigger
0,1,2,1,1,'dss.wav','',254
1,2,1,2,2,'aske_story4_trial_1.wav','marianne_story3_trial_1.wav',133
2,2,2,1,2,'aske_story4_trial_2.wav','marianne_story3_trial_2.wav',135
3,1,1,1,1,'dss.wav','',252
4,2,2,2,2,'aske_story4_trial_3.wav','marianne_story3_trial_3.wav',150


In [108]:
"""
# Estimate TRFs
# -------------
# Loop through subjects to estimate TRFs
for subject in SUBJECTS:
    subject_trf_dir = TRF_DIR / subject
    subject_trf_dir.mkdir(exist_ok=True)
    # Generate all TRF paths so we can check whether any new TRFs need to be estimated
    trf_paths = {model: subject_trf_dir / f'{subject} {model}.pickle' for model in models}
    # Skip this subject if all files already exist
    if all(path.exists() for path in trf_paths.values()):
        continue
    # Load the EEG data
    raw = mne.io.read_raw(EEG_DIR / f'{subject}' / f'{subject}-raw.fif', preload=True)
    # Band-pass filter the raw data between 0.5 and 20 Hz
    raw.filter(0.5, 20, n_jobs=1)
    # Interpolate bad channels
    raw.interpolate_bads()


    
    # Extract the events marking the stimulus presentation from the EEG file
    events = eelbrain.load.fiff.events(raw)
    # Not all subjects have all trials; determine which stimuli are present
    trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event']]
    # Extract the EEG data segments corresponding to the stimuli
    trial_durations = [durations[i] for i in trial_indexes]
    eeg = eelbrain.load.fiff.variable_length_epochs(events, -0.100, trial_durations, decim=5, connectivity='auto')
    # Since trials are of unequal length, we will concatenate them for the TRF estimation.
    eeg_concatenated = eelbrain.concatenate(eeg)



    # EEG data from cocoha already has been epoched and concatenated, so we can directly load the concatenated data
    eeg = eelbrain.load.mne.raw_ndvar(raw) # Load the raw data as NDVar 64 sensor, 192000 time


    for model, predictors in models.items():
        path = trf_paths[model]
        # Skip if this file already exists
        if path.exists():
            continue
        print(f"Estimating: {subject} ~ {model}")
        
        # Select and concetenate the predictors corresponding to the EEG trials
        # FIND ORDER OF TRIALS IN EXPINFO
        #predictors_concatenated = []
        #for predictor in predictors:
        #    predictors_concatenated.append(eelbrain.concatenate([predictor[i] for i in trial_indexes]))




        
        predictors_concatenated = []

        for predictor in predictors:

            for _, row in expinfo_table.iterrows():

                if row['n_speakers'] == 1:
                    continue

                if row['attend_mf'] == 1:
                    wavfile = row['wavfile_male']
                else:
                    wavfile = row['wavfile_female']

                path = wavfile.strip("'").removesuffix('.wav')

                predictors_concatenated.append(predictor[path])

            predictor_long = eelbrain.concatenate(predictors_concatenated)

            print("\nTotal trials concatenated:", len(predictors_concatenated))
            print("Total duration (sec):", predictor_long.time.tstop)





        
        # Fit the mTRF
        trf = eelbrain.boosting(eeg, predictors_concatenated, -0.100, 1.000, error='l1', basis=0.050, partitions=5, test=1, selective_stopping=True)
        # Save the TRF for later analysis
        eelbrain.save.pickle(trf, path)
        
"""

'\n# Estimate TRFs\n# -------------\n# Loop through subjects to estimate TRFs\nfor subject in SUBJECTS:\n    subject_trf_dir = TRF_DIR / subject\n    subject_trf_dir.mkdir(exist_ok=True)\n    # Generate all TRF paths so we can check whether any new TRFs need to be estimated\n    trf_paths = {model: subject_trf_dir / f\'{subject} {model}.pickle\' for model in models}\n    # Skip this subject if all files already exist\n    if all(path.exists() for path in trf_paths.values()):\n        continue\n    # Load the EEG data\n    raw = mne.io.read_raw(EEG_DIR / f\'{subject}\' / f\'{subject}-raw.fif\', preload=True)\n    # Band-pass filter the raw data between 0.5 and 20 Hz\n    raw.filter(0.5, 20, n_jobs=1)\n    # Interpolate bad channels\n    raw.interpolate_bads()\n\n\n\n    # Extract the events marking the stimulus presentation from the EEG file\n    events = eelbrain.load.fiff.events(raw)\n    # Not all subjects have all trials; determine which stimuli are present\n    trial_indexes = [STI

In [109]:
# Estimate TRFs
# -------------
# Loop through subjects to estimate TRFs
for subject in SUBJECTS:
    subject_trf_dir = TRF_DIR / subject
    subject_trf_dir.mkdir(exist_ok=True)
    # Generate all TRF paths so we can check whether any new TRFs need to be estimated
    trf_paths = {model: subject_trf_dir / f'{subject} {model}.pickle' for model in models}
    # Skip this subject if all files already exist
    if all(path.exists() for path in trf_paths.values()):
        continue
    # Load the EEG data
    raw = mne.io.read_raw(EEG_DIR / f'{subject}' / f'{subject}-raw.fif', preload=True)
    # Band-pass filter the raw data between 0.5 and 20 Hz
    raw.filter(0.5, 20, n_jobs=1)
    # Interpolate bad channels
    raw.interpolate_bads()


    
    # # Extract the events marking the stimulus presentation from the EEG file
    # events = eelbrain.load.fiff.events(raw)
    # # Not all subjects have all trials; determine which stimuli are present
    # trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event']]
    # # Extract the EEG data segments corresponding to the stimuli
    # trial_durations = [durations[i] for i in trial_indexes]
    # eeg = eelbrain.load.fiff.variable_length_epochs(events, -0.100, trial_durations, decim=5, connectivity='auto')
    # # Since trials are of unequal length, we will concatenate them for the TRF estimation.
    # eeg_concatenated = eelbrain.concatenate(eeg)



    # EEG data from cocoha already has been epoched and concatenated, so we can directly load the concatenated data
    eeg = eelbrain.load.mne.raw_ndvar(raw) # Load the raw data as NDVar 64 sensor, 192000 time


    for model, predictor_dicts in models.items():
        trf_path = trf_paths[model]

        if trf_path.exists():
            continue

        print(f"Estimating: {subject} ~ {model}")

        model_predictors = []  # one entry per predictor type

        # Check if predictor_dicts is a nested dict (multiple predictors) or a simple dict (single predictor)
        if isinstance(list(predictor_dicts.values())[0], dict):
            # Multiple predictors: predictor_dicts has keys like 'envelope', 'onset'
            predictor_types = predictor_dicts
        else:
            # Single predictor: wrap in a list to iterate once
            predictor_types = {'predictor': predictor_dicts}

        for predictor_name, predictor_dict in predictor_types.items():

            trial_predictors = []

            for _, row in expinfo_table.iterrows():

                if row['n_speakers'] == 1:
                    continue

                if row['attend_mf'] == 1:
                    wavfile = row['wavfile_male']
                else:
                    wavfile = row['wavfile_female']

                wavfile = wavfile.strip("'")
                path_key = wavfile.removesuffix('.wav')

                trial_predictors.append(predictor_dict[path_key])

            predictor_long = eelbrain.concatenate(trial_predictors)

            print(f"  Predictor '{predictor_name}' trials: {len(trial_predictors)}")
            print(f"  Predictor '{predictor_name}' duration: {predictor_long.time.tstop:.2f}s")

            model_predictors.append(predictor_long)

        # -------------------
        # Final sanity check
        # -------------------
        print("EEG duration:", eeg.time.tstop)
        print("Predictor duration:", model_predictors[0].time.tstop)

        # Fit TRF
        trf = eelbrain.boosting(
            eeg,
            model_predictors,
            -0.100,
            1.000,
            error='l1',
            basis=0.050,
            partitions=5,
            test=1,
            selective_stopping=True
        )

        eelbrain.save.pickle(trf, trf_path)


/var/folders/q5/dlf_55750jq9b22jkpyf6c940000gn/T/ipykernel_38337/3024059237.py:17: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


Estimating: S4_data_preproc ~ envelope
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S4_data_preproc ~ gammatone
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S4_data_preproc ~ envelope+onset
  Predictor 'envelope' trials: 60
  Predictor 'envelope' duration: 3000.00s
  Predictor 'onset' trials: 60
  Predictor 'onset' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0


/var/folders/q5/dlf_55750jq9b22jkpyf6c940000gn/T/ipykernel_38337/3024059237.py:17: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


Estimating: S3_data_preproc ~ envelope
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S3_data_preproc ~ gammatone
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S3_data_preproc ~ envelope+onset
  Predictor 'envelope' trials: 60
  Predictor 'envelope' duration: 3000.00s
  Predictor 'onset' trials: 60
  Predictor 'onset' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0


/var/folders/q5/dlf_55750jq9b22jkpyf6c940000gn/T/ipykernel_38337/3024059237.py:17: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


...

Estimating: S6_data_preproc ~ envelope
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S6_data_preproc ~ gammatone
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S6_data_preproc ~ envelope+onset
  Predictor 'envelope' trials: 60
  Predictor 'envelope' duration: 3000.00s
  Predictor 'onset' trials: 60
  Predictor 'onset' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0


/var/folders/q5/dlf_55750jq9b22jkpyf6c940000gn/T/ipykernel_38337/3024059237.py:17: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


Estimating: S10_data_preproc ~ envelope
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S10_data_preproc ~ gammatone
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S10_data_preproc ~ envelope+onset
  Predictor 'envelope' trials: 60
  Predictor 'envelope' duration: 3000.00s
  Predictor 'onset' trials: 60
  Predictor 'onset' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0


/var/folders/q5/dlf_55750jq9b22jkpyf6c940000gn/T/ipykernel_38337/3024059237.py:17: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw.interpolate_bads()


Estimating: S17_data_preproc ~ envelope
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S17_data_preproc ~ gammatone
  Predictor 'predictor' trials: 60
  Predictor 'predictor' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
Estimating: S17_data_preproc ~ envelope+onset
  Predictor 'envelope' trials: 60
  Predictor 'envelope' duration: 3000.00s
  Predictor 'onset' trials: 60
  Predictor 'onset' duration: 3000.00s
EEG duration: 3000.0
Predictor duration: 3000.0
